In [1]:
import numpy as np
import pandas as pd
import xlrd
import openpyxl

import os
folder_name = '成品仓库进出明细/'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

type_name = '成品仓库进出明细'
input_filename = type_name +'.xls'
output_filename = folder_name + type_name+ '_添加类别.xlsx'

#df = pd.read_excel(input_filename, sheetname ='成品仓库进出明细3月')
df = pd.read_excel(input_filename,sheet_name = 0)
df.columns.values


WARNING *** file size (1911956) not 512 + multiple of sector size (512)


array(['单据', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', '属性',
       'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13',
       'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', '入库统计', 'Unnamed: 18',
       '出库统计', 'Unnamed: 20'], dtype=object)

In [2]:
#清除 "入库统计"
tag_names = list(df.iloc[0])
col_names = list(df.columns.values)

idx_entering_warehouse = col_names.index('入库统计')
str_entering_warehouse_1= col_names[idx_entering_warehouse]
str_entering_warehouse_2 = col_names[idx_entering_warehouse +1]

print(str_entering_warehouse_1)
print(str_entering_warehouse_2)

df.drop([str_entering_warehouse_1,str_entering_warehouse_2], axis=1, inplace = True)


入库统计
Unnamed: 18


In [3]:
#重命名"类别名称"所在列 的标签
tag_names = list(df.iloc[0])
col_names = list(df.columns.values)

idx_typename = tag_names.index('类别名称')
str_typename = col_names[idx_typename]
print(str_typename,'\n')

df.rename(index=str, columns={str_typename: 'type_name'},inplace=True)
df.type_name.unique()

str_typesize = col_names[idx_typename + 1]
print(str_typesize, '\n')

df.rename(index=str, columns={str_typesize: 'pd_size'}, inplace = True)

# 添加一列，命名为 type
print(df.columns.values,'\n')
df.insert(idx_typename,'type',df.type_name)
df.columns.values


Unnamed: 8 

Unnamed: 9 

['单据' 'Unnamed: 1' 'Unnamed: 2' 'Unnamed: 3' 'Unnamed: 4' '属性' 'Unnamed: 6'
 'Unnamed: 7' 'type_name' 'pd_size' 'Unnamed: 10' 'Unnamed: 11'
 'Unnamed: 12' 'Unnamed: 13' 'Unnamed: 14' 'Unnamed: 15' 'Unnamed: 16'
 '出库统计' 'Unnamed: 20'] 



array(['单据', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', '属性',
       'Unnamed: 6', 'Unnamed: 7', 'type', 'type_name', 'pd_size',
       'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13',
       'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', '出库统计', 'Unnamed: 20'], dtype=object)

In [4]:
df.type[df.type_name == '类别名称'] = '类别'

df.type[(df.type_name =='HDF') | (df.type_name == 'HDG') | (df.type_name == 'HDJ') | (df.type == 'HDT') ] = '地砖'
df.type[df.type_name =='HM'] = '木纹砖'
df.type[(df.type_name =='HNF') | (df.type_name == 'HNG') | (df.type_name == 'HNJ') | (df.type == 'HNT') ] = '瓷片'
df.type[df.type_name =='HTF'] = '地砖'
df.type[df.type_name == 'HW'] = '微晶'
df.type[df.type_name == 'HY'] = '全抛釉'

df.type[df.type_name == 'PG'] ='通体砖'
df.type[(df.type_name == 'PG') & (df.pd_size == '300X300')] = '地砖'
df.type[(df.type_name == 'PG') & (df.pd_size == '300X600')] = '瓷片'

df.type[(df.type_name == 'P') | (df.type_name == 'T')] = '通体砖'
df.type[(df.type_name == 'W') | (df.type_name == 'WM')] = '瓷片'
df.type[df.type_name == 'XF'] = '地砖'
df.type[(df.type_name == 'XG') | (df.type_name == 'XT') | (df.type_name == 'XW')] = '瓷片'
df.type[df.type_name == 'H3S'] = '色晶砖'
df.type[df.type_name == 'HS'] = '大理石'
df.type[df.type_name == '浮雕'] = '浮雕'
df.type[df.type_name == '文具'] = '五金仓'
df.type[df.type_name == '花片'] = '花片'
df.type[df.type_name == '欧驰'] = '欧驰'
df.type[df.type_name == '腰线'] = '腰线'
df.type[df.type_name == '纸箱'] = '纸箱'

df.type[df.type_name == 'HNW'] = '镜面砖'

print(df.groupby('type').type.agg('count'))

type
五金仓       90
全抛釉        2
其它费用       1
办公设备       1
古宝斯        5
地砖       537
木纹砖       16
欧驰        23
浮雕        69
瓷片      3754
类别         1
纸箱         5
腰线       122
花片       175
通体砖       28
Name: type, dtype: int64


In [5]:
#modified output
output_df = df.copy(deep=True)
output_df.columns = output_df.iloc[0,:]
output_df.drop(['0'], axis=0,inplace = True)
output_df.set_index('编号',inplace = True)

writer = pd.ExcelWriter(output_filename) #creates instance of an excel workboo
output_df.to_excel(writer,'sheet1') #creates workbook 
writer.save() #saves workbook to file in python file directory

In [6]:
## regroup data
df2 = output_df.copy(deep=True)
print(df2['销售类型'].unique())
print( df2.columns.values )

['免费样板' '正常销售' '折价销售' nan '免费广告' '免费费用' '物资自用' '收费样板' '销售费用']
['日期' '类型' '销售类型' '往来单位' '仓库名称' '产品编号' '产品名称' '类别' '类别名称' '规格' '庄数' '色号'
 '等级' '品牌' '包装' '单位' '单价' '数量(片)' '金额']


In [7]:
# 销售成本 : '正常销售 + 折价销售 + 收费样板 + 销售调价

#sales = df2[ (df2['销售类型'] == '正常销售') | (df2['销售类型'] == '折价销售') | (df2['销售类型'] == '收费样板') | (df2['销售类型'] == '销售调价') ]
sales = df2[ (df2['销售类型'] != '挂账样板') & 
             (df2['销售类型'] != '挂账广告') & 
             (df2['销售类型'] != '免费广告') & 
             (df2['销售类型'] != '免费样板') &
             (df2['销售类型'] != '免费费用') ]


# 挂账样板

model = df2[ df2['销售类型'] == '挂账样板' ]

# 挂账广告

adv = df2[ df2['销售类型'] == '挂账广告']

# 免费广告

free_adv = df2[ df2['销售类型'] == '免费广告']

# 免费样板

free_model = df2[ df2['销售类型'] == '免费样板']

# 免费费用

free_expenses = df2[ df2['销售类型'] == '免费费用']


In [8]:
def summary_product_category(key, df_temp):
    output_filename = folder_name +'summary_'+key+'.xlsx'
    output_filename2 = folder_name +'往来单位_'+key+'.xlsx'


    group = pd.concat([ df_temp.groupby('类别')['数量(片)'].agg(np.sum),
                        df_temp.groupby('类别')['金额'].agg(np.sum)],
                       axis = 1)

    group.loc[group.shape[0]] = group.sum()
    group.rename(index = {group.shape[0]-1: '总计'}, inplace = True)

    writer = pd.ExcelWriter(output_filename) #creates instance of an excel workboo
    group.to_excel(writer,startrow = 1, sheet_name= 'sheet1') #creates workbook

    # handle the spreadsheet format
    workbook = writer.book
    worksheet = writer.sheets['sheet1']      
    percent_fmt = workbook.add_format({'num_format': '0.00'})
    worksheet.set_column('C:C', None, percent_fmt)
    header_format = workbook.add_format({'bold': True,'text_wrap': True,
                                                 'align': 'center',
                                                 'valign': 'vcenter',
                                                 'fg_color': '#D7E4BC', 'border': 1})
    worksheet.merge_range('A1:C1',key,header_format)        

    writer.save() #saves workbook to file in python file directory

    ########################
    # 往来单位
    ########################
    group2 = df_temp.groupby('往来单位')['金额'].agg(np.sum).reset_index().set_index('往来单位')

    group2.loc[group2.shape[0]] = group2.sum()
    group2.rename(index = {group2.shape[0]-1: '总计'}, inplace = True)

    writer = pd.ExcelWriter(output_filename2) #creates instance of an excel workboo
    group2.to_excel(writer,startrow = 1, sheet_name = 'sheet1') #creates workbook 

    # handle the spreadsheet format
    workbook = writer.book
    worksheet = writer.sheets['sheet1']      
    percent_fmt = workbook.add_format({'num_format': '0.00'})
    worksheet.set_column('B:B', None, percent_fmt)
    header_format = workbook.add_format({'bold': True,'text_wrap': True,
                                                 'align': 'center',
                                                 'valign': 'vcenter',
                                                 'fg_color': '#D7E4BC', 'border': 1})
    worksheet.merge_range('A1:B1', key+'往来单位',header_format)        

    writer.save() #saves workbook to file in python file directory
    
    return



In [9]:
key = '销售成本'
summary_product_category(key, sales)

key ='挂账样板'
summary_product_category(key, model)

key = '免费样板'
summary_product_category(key, free_model)

key = '免费广告'
summary_product_category(key, free_adv)

key = '免费费用'
summary_product_category(key, free_expenses)


In [10]:
# 不同销售类型的金额汇总

output_filename = folder_name + '不同销售类型金额汇总.xlsx'
group = df2.groupby('销售类型')['金额'].agg(np.sum)

writer = pd.ExcelWriter(output_filename) #creates instance of an excel workboo
group.to_excel(writer,startrow = 1, sheet_name= 'sheet1') #creates workbook

# handle the spreadsheet format
workbook = writer.book
worksheet = writer.sheets['sheet1']      
percent_fmt = workbook.add_format({'num_format': '0.00'})
worksheet.set_column('B:B', None, percent_fmt)
header_format = workbook.add_format({'bold': True,'text_wrap': True,
                                             'align': 'center',
                                             'valign': 'vcenter',
                                             'fg_color': '#D7E4BC', 'border': 1})
worksheet.merge_range('A1:B1','销售类型金额统计',header_format)        

writer.save() #saves workbook to file in python file directory


In [11]:
print('success')

success
